In [16]:
# create iteration for all states from 2017 to 2018 year
import json
import requests
import pandas as pd
import re



def run_dataset(url):

# Call data from API
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
                }
   
    #url = "https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states=1&fromYear=2016&toYear=2017&minNumOfVehicles=1&maxNumOfVehicles=6&format=json"
    json_data = requests.get(url, headers=headers).json()
    json_data.keys()
    results= json_data['Results'][0]
    

# Convert json format to dataframe
    cols = results[0].keys() 
    vals = [data.values() for data in results]
    Crash_list = pd.DataFrame(vals,columns=cols)
    
    
#Create FIPS code
    def get_num(x):
        num = re.findall(r'[(](.*?)[)]', x)[0]
        return num

    def transfer_County(num):
        if len(num) < 3:
            num = '0' + num
        elif len(num) < 2:
            num = '00'+ num
        else:
            pass
        return num
    def transfer_State(num):
        if len(str(num))<2:
            num = '0' + str(num)
        return num

    Crash_list['County_num'] = Crash_list['CountyName'].apply(get_num)
    Crash_list['New_County_num'] = Crash_list['County_num'].apply(transfer_County)
    Crash_list['New_State_num'] = Crash_list['State'].apply(transfer_State)
    Crash_list['FIPS_Code'] = Crash_list['New_State_num'] + Crash_list['New_County_num']
    Crash_list = Crash_list.drop(['New_County_num','New_State_num'],axis=1)
    
#loop crashdate column
    Crash_list['Crash_time'] = 0
    for i in range(0,len(Crash_list)):
        try:
            m = re.search(r"Date\((\d+)\-", Crash_list['CrashDate'][i].strip('/'))
            Crash_list.loc[i, 'Crash_time'] = datetime.fromtimestamp(int(m.group(1))/1000.0).strftime('%Y-%m-%d %H:%M:%S')
        except:
            Crash_list.loc[i, 'Crash_time'] = None
        
    
    
#Create a new 'County' column and loop on 'CountyName' column
    Crash_list['County'] = Crash_list['CountyName'].str.split(' ').str[0]
    

# Remove the junk columns and rearrange the column order
    a=Crash_list.drop(columns=['CountyName','CrashDate'])
    Crash_List=a[['St_Case','State','StateName','County_num','County','FIPS_Code','Crash_time','Fatals','Peds','Persons','TotalVehicles']]


    Crash_List.to_csv("State Number %s." %stateid)

In [65]:
URLhead = "https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states="
URLMiddle1 = "&fromYear="
URLMiddle2 = "&toYear="
URLTail = "&minNumOfVehicles=1&maxNumOfVehicles=6&format=json"

URLlistTail = list()
for i,j in zip(range(2016,2020),range(2017,2021)):
    individualURLTail = URLMiddle1+str(i)+URLMiddle2+str(j)+URLTail
    URLlistTail.append(individualURLTail)

URLlist = list()
for URLtail in URLlistTail:
    for i in range(1,51):
        individualURL = URLhead+str(i)+URLtail
        URLlist.append(individualURL)
    

In [66]:
#URLlistTail
#del URLlist[2]
#URLlist
URLlist
#len(URLlist)

['https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states=1&fromYear=2016&toYear=2017&minNumOfVehicles=1&maxNumOfVehicles=6&format=json',
 'https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states=2&fromYear=2016&toYear=2017&minNumOfVehicles=1&maxNumOfVehicles=6&format=json',
 'https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states=3&fromYear=2016&toYear=2017&minNumOfVehicles=1&maxNumOfVehicles=6&format=json',
 'https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states=4&fromYear=2016&toYear=2017&minNumOfVehicles=1&maxNumOfVehicles=6&format=json',
 'https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states=5&fromYear=2016&toYear=2017&minNumOfVehicles=1&maxNumOfVehicles=6&format=json',
 'https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states=6&fromYear=2016&toYear=2017&minNumOfVehicles=1&maxNumOfVehicles=6&format=json',
 'https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/GetCaseList?states=7&fromYear=2016&

In [34]:
stateid = 0

for url in URLlist:
    run_dataset(url)
    #save every state's dataframe to csv
    stateid = stateid +1
    Crash_List.to_csv("State Number %s.csv" %str(stateid))

IndexError: list index out of range